# CPE KMUTT website scraper

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import re
import pandas as pd
#from selenium.webdriver.edge.options import Options

In [ ]:
driver = webdriver.Edge(executable_path = 'edgedriver_win64\\msedgedriver.exe')

<h2> Name, Position, and URLs </h2>

In [ ]:
url = 'https://www.cpe.kmutt.ac.th/th/staff'
driver.get(url)
sleep(0.5)
# get HTML from driver at current page
soup = BeautifulSoup(driver.page_source,'html.parser')
# declare lists
position = []
postid = []
fullname = []
for i in range(0, len(soup.select('p.text-muted'))):
    position.append(soup.select('p.text-muted')[i].get_text())
    fullname.append(soup.select('h3.profile-username')[i].get_text())
    tmp = re.search(r'(postid)=\"\S+\"', str(soup.select('h3.profile-username')[i]))
    postid.append(tmp.group(0)[8:-1])

personnel = pd.DataFrame({'PostID': postid,
                          'Position': position,
                          'FullName': fullname})

## Education and Research field

In [ ]:
# Use .format to fill url for the best time-complexity
urls = url + "-detail/{}"
education = list()
role, tel, email = [],[],[] 
for idx in postid:
    driver.get(urls.format(idx))
    sleep(2)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    info = soup.select('p.staff-list-detail')
    edu = soup.select('div.col-xs-12.post-content')[0].get_text()
    education.append(edu)
    role.append(info[0].get_text())
    tel.append(info[1].get_text())
    email.append(info[2].get_text())

moreinfo = pd.DataFrame({'role':role, 
                         'tel':tel,
                         'email':email,
                         'education':education})

## Constructing DataFrame

In [ ]:
df = pd.concat([personnel, moreinfo], axis = 1)
#df

In [ ]:
test = df.iloc[9,3].split('\n')
list(filter(lambda x: x != '', test))